In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)
type(review_data)

list

In [5]:
import random
random_data = random.sample(review_data,len(review_data))

In [6]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df=0.3 ,min_df = 0.004, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)

In [8]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

In [9]:
from sklearn import cross_validation

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)

In [11]:
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83868, 2504), (83868,), (9316, 2504), (9316,))

BaggingClassifier

In [13]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())

0.573636612992


RandomForestClassifier

In [14]:
from sklearn.ensemble import RandomForestClassifier
seed = 7
num_trees = 100
max_features = 3
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())

0.561611608728


ExtraTreesClassifier

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

seed = 7
num_trees = 100
max_features = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())

0.562148780904


AdaBoostClassifier

In [16]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())

0.571704821527


GradientBoostingClassifier

In [17]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

Voting

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold)
print(results.mean())